### 2-site Kitaev chain

In [9]:
# IMPORTANT: Ensure this entire script is run, not just a partial selection.
# A "SyntaxError: unexpected EOF while parsing" can occur if an incomplete
# portion of the script is executed, as the Python parser would expect more code.

import matplotlib # Import matplotlib first
matplotlib.use('TkAgg') # Set the backend BEFORE importing pyplot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from scipy.linalg import expm
import traceback # Added for detailed error reporting

# Define gates
I = np.array([[1, 0], [0, 1]], dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)
ket_0 = np.array([[1], [0]], dtype=complex) # ket |0>
ket_1 = np.array([[0], [1]], dtype=complex) # ket |1>

# Define Initial States
initial_state_bell = (1/np.sqrt(2))*np.kron(ket_0,ket_0) + (1/np.sqrt(2))*np.kron(ket_1,ket_1)
initial_state_10 = np.kron(ket_1,ket_0)

# Time vector for plotting (user specified 600 points)
time_vec = np.linspace(0, 4 * np.pi, 600)

# Operators for expectation values
n_is1_op = (1/2)*(np.kron(I, I) - np.kron(Z, I))
n_is2_op = (1/2)*(np.kron(I, I) - np.kron(I, Z)) 
cc_op_op = (-0.25*np.kron(X,X)-0.25j*np.kron(X,Y)-0.25j*np.kron(Y,X) + 0.25*np.kron(Y,Y))
cc_dagger_op_op = 0.25*(np.kron(X,X)-0.25j*np.kron(X,Y)-0.25*np.kron(Y,X) -0.25*np.kron(Y,Y)) 
gamma_op_op = 1j*(np.kron(Z,I))

# Function to calculate Hamiltonian based on current slider values
def calculate_hamiltonian(t_val, eps_val, delta_val):
    return (eps_val/2)*(np.kron(I, I) - np.kron(I, Z)) \
           + (eps_val/2)*(np.kron(I, I) - np.kron(Z, I)) \
           + ((t_val+delta_val)/2)*np.kron(X, X) \
           + ((t_val-delta_val)/2)*np.kron(Y, Y)

# Function to calculate expectation values over time for a given Hamiltonian and initial state
def calculate_dynamics(H_analytical_current, initial_state):
    n1_vals, n2_vals, c_vals, c_dag_vals, gamma_vals, E_vals = [], [], [], [], [], [] 
    for time_step in time_vec:
        U_analytical = expm(-1.0j * H_analytical_current * time_step)
        current_final_state = U_analytical @ initial_state
        
        n1_vals.append(np.abs(current_final_state.conj().T @ n_is1_op @ current_final_state)[0,0])
        n2_vals.append(np.abs(current_final_state.conj().T @ n_is2_op @ current_final_state)[0,0]) 
        c_vals.append(np.abs(current_final_state.conj().T @ cc_op_op @ current_final_state)[0,0])
        c_dag_vals.append(np.abs(current_final_state.conj().T @ cc_dagger_op_op @ current_final_state)[0,0]) 
        gamma_vals.append(np.abs(current_final_state.conj().T @ gamma_op_op @ current_final_state)[0,0])
        E_vals.append(np.abs(current_final_state.conj().T @ H_analytical_current @ current_final_state)[0,0]) 
    return n1_vals, n2_vals, c_vals, c_dag_vals, gamma_vals, E_vals

# Fixed Y-axis limits
y_limits_fixed = [-0.05, 1.05] # For non-energy plots
y_limits_energy = [-1, 10]    # For energy plots

# Initial parameters
initial_t = 1.0
initial_eps = 0.0
initial_delta = 1.0

# --- Figure 1: Initial State |10> ---
fig1, axs1 = plt.subplots(3, 2, figsize=(12, 10)) 
fig1.subplots_adjust(left=0.1, bottom=0.30, right=0.95, top=0.92, hspace=0.4, wspace=0.3) 

# Initial calculation for plots
H_initial = calculate_hamiltonian(initial_t, initial_eps, initial_delta)
n1_10_vals, n2_10_vals, c_10_vals, c_dag_10_vals, gamma_10_vals, E_10_vals = calculate_dynamics(H_initial, initial_state_10)

# Plotting for |10> state (fig1)
# Column 1
line_n1_10, = axs1[0, 0].plot(time_vec, n1_10_vals, label=r'<$n_1$>')
axs1[0, 0].set_ylabel('Expectation Value') 
axs1[0, 0].set_title('Site 1 occupation <n₁>')
axs1[0, 0].grid(True); axs1[0, 0].legend(); axs1[0, 0].set_ylim(y_limits_fixed)

line_c_10, = axs1[1, 0].plot(time_vec, c_10_vals, label='|<cc>|')
axs1[1, 0].set_ylabel('Expectation Value') 
axs1[1, 0].set_title('Pair-creation operator |<cc>|')
axs1[1, 0].grid(True); axs1[1, 0].legend(); axs1[1, 0].set_ylim(y_limits_fixed)

line_gamma_10, = axs1[2, 0].plot(time_vec, gamma_10_vals, label=r'|<$\gamma$>|')
axs1[2, 0].set_xlabel('Time') 
axs1[2, 0].set_ylabel('Expectation Value') 
axs1[2, 0].set_title(r'Gamma operator |<$\gamma$>|')
axs1[2, 0].grid(True); axs1[2, 0].legend(); axs1[2, 0].set_ylim(y_limits_fixed)

# Column 2
line_n2_10, = axs1[0, 1].plot(time_vec, n2_10_vals, label=r'<$n_2$>')
axs1[0, 1].set_title('Site 2 occupation <n₂>')
axs1[0, 1].grid(True); axs1[0, 1].legend(); axs1[0, 1].set_ylim(y_limits_fixed)

line_c_dag_10, = axs1[1, 1].plot(time_vec, c_dag_10_vals, label=r'|<$c^\dagger c^\dagger$>|')
axs1[1, 1].set_title(r'Pair-annihilation operator |<$c^\dagger c^\dagger$>|')
axs1[1, 1].grid(True); axs1[1, 1].legend(); axs1[1, 1].set_ylim(y_limits_fixed)

line_E_10, = axs1[2, 1].plot(time_vec, E_10_vals, label='|<E>|')
axs1[2, 1].set_xlabel('Time')
axs1[2, 1].set_title('Energy |<H>|')
axs1[2, 1].grid(True); axs1[2, 1].legend(); axs1[2, 1].set_ylim(y_limits_energy) # Use energy limits

fig1_title = fig1.suptitle(rf'Expectation Values for Initial State |10> (t={initial_t:.2f}, $\varepsilon$={initial_eps:.2f}, $\Delta$={initial_delta:.2f})', fontsize=16)

# --- Figure 2: Bell Initial State ---
fig2, axs2 = plt.subplots(3, 2, figsize=(12, 10)) 
fig2.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.92, hspace=0.4, wspace=0.3)

# Initial calculation for Bell state plots
n1_bell_vals, n2_bell_vals, c_bell_vals, c_dag_bell_vals, gamma_bell_vals, E_bell_vals = calculate_dynamics(H_initial, initial_state_bell)

# Plotting for Bell state (fig2)
# Column 1
line_n1_bell, = axs2[0, 0].plot(time_vec, n1_bell_vals, label=r'<$n_1$>')
axs2[0, 0].set_ylabel('Expectation Value') 
axs2[0, 0].set_title('Site 1 occupation <n₁>')
axs2[0, 0].grid(True); axs2[0, 0].legend(); axs2[0, 0].set_ylim(y_limits_fixed)

line_c_bell, = axs2[1, 0].plot(time_vec, c_bell_vals, label='|<cc>|')
axs2[1, 0].set_ylabel('Expectation Value') 
axs2[1, 0].set_title(r'Pair-creation operator |<cc>|')
axs2[1, 0].grid(True); axs2[1, 0].legend(); axs2[1, 0].set_ylim(y_limits_fixed)

line_gamma_bell, = axs2[2, 0].plot(time_vec, gamma_bell_vals, label=r'|<$\gamma$>|')
axs2[2, 0].set_xlabel('Time')
axs2[2, 0].set_ylabel('Expectation Value') 
axs2[2, 0].set_title(r'Gamma operator |<$\gamma \gamma$>|') 
axs2[2, 0].grid(True); axs2[2, 0].legend(); axs2[2, 0].set_ylim(y_limits_fixed)

# Column 2
line_n2_bell, = axs2[0, 1].plot(time_vec, n2_bell_vals, label=r'<$n_2$>')
axs2[0, 1].set_title('Site 2 occupation <n₂>')
axs2[0, 1].grid(True); axs2[0, 1].legend(); axs2[0, 1].set_ylim(y_limits_fixed)

line_c_dag_bell, = axs2[1, 1].plot(time_vec, c_dag_bell_vals, label=r'|<$c^\dagger c^\dagger$>|')
axs2[1, 1].set_title(r'Pair-annihilation operator |<$c^\dagger c^\dagger$>|')
axs2[1, 1].grid(True); axs2[1, 1].legend(); axs2[1, 1].set_ylim(y_limits_fixed)

line_E_bell, = axs2[2, 1].plot(time_vec, E_bell_vals, label='|<E>|')
axs2[2, 1].set_xlabel('Time')
axs2[2, 1].set_title('Energy |<H>|')
axs2[2, 1].grid(True); axs2[2, 1].legend(); axs2[2, 1].set_ylim(y_limits_energy) # Use energy limits

fig2_title = fig2.suptitle(rf'Expectation Values for Bell Initial State (t={initial_t:.2f}, $\varepsilon$={initial_eps:.2f}, $\Delta$={initial_delta:.2f})', fontsize=16)

# --- Sliders (associated with fig1) ---
ax_slider_t = fig1.add_axes([0.15, 0.18, 0.7, 0.03])
ax_slider_eps = fig1.add_axes([0.15, 0.13, 0.7, 0.03])
ax_slider_delta = fig1.add_axes([0.15, 0.08, 0.7, 0.03])

s_t = Slider(ax_slider_t, 't (Hopping)', 0.0, 5.0, valinit=initial_t, valstep=0.1)
s_eps = Slider(ax_slider_eps, r'$\varepsilon$ (On-site E)', -2.0, 2.0, valinit=initial_eps, valstep=0.1) 
s_delta = Slider(ax_slider_delta, r'$\Delta$ (Asymmetry)', -2.0, 2.0, valinit=initial_delta, valstep=0.1) 

# Update function to be called when sliders change
def update(val):
    print(f"Update function called. Slider values: t={s_t.val:.2f}, ε={s_eps.val:.2f}, δ={s_delta.val:.2f}")
    try:
        current_t = s_t.val
        current_eps = s_eps.val
        current_delta = s_delta.val
        
        H_current = calculate_hamiltonian(current_t, current_eps, current_delta)
        
        # Recalculate dynamics for |10> state
        n1_10, n2_10, c_10, c_dag_10, gamma_10, E_10 = calculate_dynamics(H_current, initial_state_10)
        line_n1_10.set_ydata(n1_10)
        line_n2_10.set_ydata(n2_10)
        line_c_10.set_ydata(c_10)
        line_c_dag_10.set_ydata(c_dag_10)
        line_gamma_10.set_ydata(gamma_10)
        line_E_10.set_ydata(E_10)
        
        # Set y-limits for fig1 plots
        for i in range(3):
            for j in range(2):
                if i == 2 and j == 1: # This is the Energy plot axs1[2,1]
                    axs1[i,j].set_ylim(y_limits_energy) # Use energy limits
                else:
                    axs1[i,j].set_ylim(y_limits_fixed)
        
        # Recalculate dynamics for Bell state
        n1_b, n2_b, c_b, c_dag_b, gamma_b, E_b = calculate_dynamics(H_current, initial_state_bell)
        line_n1_bell.set_ydata(n1_b)
        line_n2_bell.set_ydata(n2_b)
        line_c_bell.set_ydata(c_b)
        line_c_dag_bell.set_ydata(c_dag_b)
        line_gamma_bell.set_ydata(gamma_b)
        line_E_bell.set_ydata(E_b)

        # Set y-limits for fig2 plots
        for i in range(3):
            for j in range(2):
                if i == 2 and j == 1: # This is the Energy plot axs2[2,1]
                    axs2[i,j].set_ylim(y_limits_energy) # Use energy limits
                else:
                    axs2[i,j].set_ylim(y_limits_fixed)
                
        # Update figure titles
        title_text = rf'(t={current_t:.2f}, $\varepsilon$={current_eps:.2f}, $\Delta$={current_delta:.2f})' 
        fig1_title.set_text(f'Expectation Values for Initial State |10> {title_text}')
        fig2_title.set_text(f'Expectation Values for Bell Initial State {title_text}')

        fig1.canvas.draw_idle()
        fig2.canvas.draw_idle()
        print("Update completed successfully.")
    except Exception as e:
        print(f"Error during update: {e}")
        traceback.print_exc()

s_t.on_changed(update)
s_eps.on_changed(update)
s_delta.on_changed(update)

plt.show()


In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from ipywidgets import interact, FloatSlider

# Define gates
I = np.array([[1, 0], [0, 1]], dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Define initial state (Bell state)
ket_0 = np.array([[1], [0]], dtype=complex)
ket_1 = np.array([[0], [1]], dtype=complex)

# Define observables
n_is1 = (1/2)*(np.kron(I, I) - np.kron(Z, I))
n_is2 = (1/2)*(np.kron(I, I) - np.kron(I, Z))
cc_op = 1/4*(-np.kron(X,X) - 1j*np.kron(X,Y) - 1j*np.kron(Y,X) + np.kron(Y,Y))
cc_dagger_op = 1/4*(np.kron(X,X) - 1j*np.kron(X,Y) - 1j*np.kron(Y,X) - np.kron(Y,Y))
def compute_observables(eps, t, delta, initial_state):
    times = np.linspace(0, 8 * np.pi, 300)
    n1_vals, n2_vals, c_vals, c_dagger_vals, E_vals = [], [], [], [], []

    for time in times:
        H = (eps/2)*(np.kron(I, I) - np.kron(I, Z)) \
          + (eps/2)*(np.kron(I, I) - np.kron(Z, I)) \
          + ((t + delta)/2)*np.kron(X, X) \
          + ((t - delta)/2)*np.kron(Y, Y)
        
        U = expm(-1.0j * H * time)
        final_state = U @ initial_state

        n1 = np.real((final_state .conj().T @ n_is1 @ final_state )[0, 0])
        n2 = np.real((final_state .conj().T @ n_is2 @ final_state )[0, 0])
        c = (final_state .conj().T @ cc_op @ final_state )[0, 0]
        #[PauliSumOp(SparsePauliOp(['XY', 'YY', 'XX', 'YX'],
               #coeffs=[ 0.  -0.25j, -0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j]), coeff=1.0)]
        c_dagger = (final_state .conj().T @ cc_dagger_op @ final_state )[0, 0]
        E = np.real((final_state .conj().T @ H @ final_state )[0, 0])

        n1_vals.append(n1)
        n2_vals.append(n2)
        c_vals.append(c)
        c_dagger_vals.append(c_dagger)
        E_vals.append(E)
    
    return times, n1_vals, n2_vals, c_vals, c_dagger_vals, E_vals

@interact(
    eps=FloatSlider(min=0, max=20, step=0.5, value=10, description='ε'),
    t=FloatSlider(min=0, max=10, step=0.5, value=5, description='t'),
    delta=FloatSlider(min=-5, max=5, step=0.5, value=2, description='Δ')
)
def plot_observables(eps, t, delta):
    initial_state_bell = (1/np.sqrt(2)) * np.kron(ket_0, ket_0) + (1/np.sqrt(2)) * np.kron(ket_1, ket_1)
    initial_state_10 = np.kron(ket_0, ket_1)#(1/np.sqrt(2)) * np.kron(ket_1, ket_0) + (1/np.sqrt(2)) * np.kron(ket_0, ket_1)

    # Compute for both states
    times, n1_bell, n2_bell, c_bell, c_dagger_bell, E_bell = compute_observables(eps, t, delta, initial_state_bell)
    _,     n1_10,   n2_10,   c_10, c_dagger_10,   E_10  = compute_observables(eps, t, delta, initial_state_10)

    fig, axs = plt.subplots(5, 2, figsize=(14, 10), sharex=True)

    # Bell state plots (left column)
    axs[0, 0].plot(times, n1_bell)
    axs[0, 0].set_ylabel('⟨n₁⟩')
    axs[0, 0].set_title(r'$|\psi\rangle = (|00\rangle + |11\rangle)/\sqrt{2}$')
    axs[0, 0].grid(True)

    axs[1, 0].plot(times, n2_bell)
    axs[1, 0].set_ylabel('⟨n₂⟩')
    axs[1, 0].grid(True)

    axs[2, 0].plot(times, c_bell, color='orange')
    axs[2, 0].set_ylabel('⟨cc⟩')
    axs[2, 0].grid(True)

    axs[3, 0].plot(times, c_dagger_bell, color='red')
    axs[3, 0].set_ylabel('⟨c+c+⟩')
    axs[3, 0].grid(True)

    axs[4, 0].plot(times, E_bell, color='green')
    axs[4, 0].set_ylabel('⟨H⟩')
    axs[4, 0].set_xlabel('Time')
    axs[4, 0].grid(True)

    # |10⟩ + |01⟩ state plots (right column)
    axs[0, 1].plot(times, n1_10)
    axs[0, 1].set_title(r'$|\psi\rangle = |01\rangle$')#.set_title(r'$|\psi\rangle = (|10\rangle + |01\rangle)/\sqrt{2}$')
    axs[0, 1].grid(True)

    axs[1, 1].plot(times, n2_10)
    axs[1, 1].grid(True)

    axs[2, 1].plot(times, c_10, color='orange')
    axs[2, 1].grid(True)

    axs[3, 1].plot(times, c_dagger_10, color='red')
    axs[3, 1].set_ylabel('⟨c+c+⟩')
    axs[3, 1].grid(True)

    axs[4, 1].plot(times, E_10, color='green')
    axs[4, 1].set_xlabel('Time')
    axs[4, 1].grid(True)

    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=10.0, description='ε', max=20.0, step=0.5), FloatSlider(value=5.0, des…

### 3-site Kitaev chain

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from ipywidgets import interact, FloatSlider

# Define gates
I = np.array([[1, 0], [0, 1]], dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Define initial state (Bell state)
ket_0 = np.array([[1], [0]], dtype=complex)
ket_1 = np.array([[0], [1]], dtype=complex)
initial_state =  np.kron(np.kron(ket_0, ket_0), ket_0) #+ np.kron(np.kron(ket_1, ket_1), ket_1)#np.kron(np.kron(ket_1, ket_1), ket_1) #(1/np.sqrt(2)) * np.kron(ket_0, ket_0) + (1/np.sqrt(2)) * np.kron(ket_1, ket_1)
                
# Define observables
n_is1 = (1/2)*(np.kron(np.kron(I, I), I) - np.kron(np.kron(Z, I), I))
n_is2 = (1/2)*(np.kron(np.kron(I, I), I) - np.kron(np.kron(I, Z), I))
n_is3 = (1/2)*(np.kron(np.kron(I, I), I) - np.kron(np.kron(I, I), Z))
cc_op = 1/4*(-np.kron(np.kron(X,Z),X) - 1j*np.kron(np.kron(X,Z),Y) - 1j*np.kron(np.kron(Y,Z),X) + np.kron(np.kron(Y,Z),Y))

def compute_observables(eps, t, delta):
    times = np.linspace(0, 8 * np.pi, 300)
    n1_vals, n2_vals, n3_vals, c_vals, E_vals = [], [], [], [], []

    for time in times:
        H = (eps/2)*(np.kron(np.kron(I, I), I) - np.kron(np.kron(Z, I), I)) \
          + (eps/2)*(np.kron(np.kron(I, I), I) - np.kron(np.kron(I, Z), I)) \
          + (eps/2)*(np.kron(np.kron(I, I), I) - np.kron(np.kron(I, I), Z)) \
          + ((t + delta)/2)*np.kron(np.kron(X, X), I) \
          + ((t - delta)/2)*np.kron(np.kron(Y, Y), I) \
          + ((t + delta)/2)*np.kron(np.kron(I, X), X) \
          + ((t - delta)/2)*np.kron(np.kron(I, Y), Y)
        
        U = expm(-1.0j * H * time)
        
        final_state = U @ initial_state

        n1 = np.real((final_state .conj().T @ n_is1 @ final_state )[0, 0])
        n2 = np.real((final_state .conj().T @ n_is2 @ final_state )[0, 0])
        n3 = np.real((final_state .conj().T @ n_is3 @ final_state )[0, 0])
        c = np.real((final_state .conj().T @ cc_op @ final_state )[0, 0])
        E = np.real((final_state .conj().T @ H @ final_state )[0, 0])
        n1_vals.append(n1)
        n2_vals.append(n2)
        n3_vals.append(n3)
        c_vals.append(c)
        E_vals.append(E)
    
    return times, n1_vals, n2_vals, n3_vals, c_vals, E_vals

@interact(
    eps=FloatSlider(min=0, max=20, step=0.5, value=10, description='ε'),
    t=FloatSlider(min=0, max=10, step=0.5, value=5, description='t'),
    delta=FloatSlider(min=-5, max=5, step=0.5, value=2, description='Δ')
)
def plot_observables(eps, t, delta):
    times, n1_vals, n2_vals, n3_vals, c_vals, E_vals = compute_observables(eps, t, delta)

    plt.figure(figsize=(12, 8))

    plt.subplot(3, 1, 1)
    plt.plot(times, n1_vals, label='⟨n_1⟩', color='red')
    plt.plot(times, n2_vals, label='⟨n_2⟩', color='blue')
    plt.plot(times, n3_vals, label='⟨n_3⟩', color='green')
    plt.legend()
    plt.ylim(0,2)
    plt.ylabel('⟨n_2⟩')
    plt.grid(True)

    plt.subplot(3, 1, 2)
    plt.plot(times, c_vals, label='⟨c_corr⟩', color='orange')
    plt.ylabel('⟨c_1c_3⟩')
    plt.grid(True)

    plt.subplot(3, 1, 3)
    plt.plot(times, E_vals, label='⟨H⟩', color='green')
    plt.xlabel('Time')
    plt.ylabel('⟨H⟩')
    plt.ylim(20,40)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=10.0, description='ε', max=20.0, step=0.5), FloatSlider(value=5.0, des…